# Handling Missing Data

In [1]:
import findspark

In [2]:
findspark.init('/home/aditya/spark-3.1.1-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("missingval").getOrCreate()

In [6]:
df = spark.read.csv("ContainsNull.csv",inferSchema = True, header = True)

In [7]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [10]:
## treat null value

## drop all rows having missing values
df.na.drop().show()

## dfault is any
df.na.drop(how = 'any').show()


+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [14]:
## drop all rows having more than equal to 2 missing values
df.na.drop(thresh = 2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [16]:
## drop only when all values in a row is null
df.na.drop(how = 'all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [17]:
## drop only when a specific column have null values
df.na.drop(subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [21]:
## fill null values
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [19]:
## spark fill based on variable type i.e. in string column fills string and in integer columns integer
df.na.fill('FILL Value').show()

+----+----------+-----+
|  Id|      Name|Sales|
+----+----------+-----+
|emp1|      John| null|
|emp2|FILL Value| null|
|emp3|FILL Value|345.0|
|emp4|     Cindy|456.0|
+----+----------+-----+



In [20]:
df.na.fill(0).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [22]:
## fill in specific columns

df.na.fill('no name', subset=['Name']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|no name| null|
|emp3|no name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



In [23]:
## fill values with mean value

from pyspark.sql.functions import mean

In [28]:
meanvalue = df.select(mean(df['Sales'])).collect()

In [32]:
mean_sales = meanvalue[0][0]

In [33]:
df.na.fill(mean_sales, subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

